## Dataset trainticket m9dgg

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
DATASET_ID = "m9dgg"

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [4]:
import sys
sys.path.append('../../')

from tsdr import tsdr
from eval import groundtruth
from meltria import loader
from eval import validation

In [7]:
metrics_files = !find "/datasets/argowf-chaos-{DATASET_ID}/" -type f -name "*.json"
records = loader.load_dataset(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=4*45, # 45min
)

In [8]:
len(records)

93

In [9]:
well_injected_records = validation.find_records_detected_anomalies_of_sli(records, faulty_datapoints=4*5, sli_index=1)

In [10]:
len(well_injected_records)

42

In [11]:
well_injected_records = validation.find_records_detected_anomalies_of_cause_metrics(well_injected_records, faulty_datapoints=4*5)

In [12]:
len(well_injected_records)

42

In [13]:
import sys
sys.path.append("../")
from eval.tsdr import sweep_tsdr_and_save_as_cache

In [14]:
import os

os.environ["TSDR_NEPTUNE_PROJECT"] = "yuuk1/tsdr"

sweep_tsdr_and_save_as_cache(
    dataset_id=DATASET_ID,
    records=well_injected_records,
    # use_manually_selected_metrics=[True, False],
    use_manually_selected_metrics=[False],
    list_of_tsdr_options=[
        # {
        #     "enable_unireducer": False,
        #     "enable_multireducer": False,
        # },
        # {
        #     "enable_unireducer": True,
        #     "enable_multireducer": False,
        #     "step1_method_name": "residual_integral",
        #     "step1_residual_integral_threshold": 20,
        #     "step1_residual_integral_change_start_point": False,
        #     "step1_residual_integral_change_start_point_n_sigma": 3,
        # },
        {
            "enable_unireducer": False,
            "enable_multireducer": True,
            "step2_clustering_method_name": "dbscan",
            "step2_dbscan_min_pts": 2,
            "step2_dbscan_dist_type": "sbd",
            "step2_dbscan_algorithm": "hdbscan",
            "step2_clustering_series_type": "raw",
            "step2_clustering_choice_method": "medoid",
        },
        {
            "enable_unireducer": True,
            "enable_multireducer": True,
            "step1_method_name": "residual_integral",
            "step1_residual_integral_threshold": 20,
            "step1_residual_integral_change_start_point": False,
            "step1_residual_integral_change_start_point_n_sigma": 3,
            "step2_clustering_method_name": "dbscan",
            "step2_dbscan_min_pts": 2,
            "step2_dbscan_dist_type": "sbd",
            "step2_dbscan_algorithm": "hdbscan",
            "step2_clustering_series_type": "raw",
            "step2_clustering_choice_method": "medoid",
        }, {
            "enable_unireducer": True,
            "enable_multireducer": True,
            "step1_method_name": "residual_integral",
            "step1_residual_integral_threshold": 20,
            "step1_residual_integral_change_start_point": False,
            "step1_residual_integral_change_start_point_n_sigma": 3,
            "step2_clustering_method_name": "hierarchy",
            "step2_hierarchy_dist_threshold": 0.02,  # should be <1.0 if 'sbd' is specified
            "step2_hierarchy_dist_type": "sbd", # 'sbd' or 'hamming'
            "step2_hierarchy_linkage_method": "single",  # 'single','complete','average','weighted', 'centroid', 'median', 'ward'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        }, 
        {
            "enable_unireducer": False,
            "enable_multireducer": True,
            "step2_clustering_method_name": "dbscan",
            "step2_dbscan_min_pts": 1,
            "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
            "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        }, 
        {
            "enable_unireducer": True,
            "enable_multireducer": True,
            "step1_method_name": "residual_integral",
            "step1_residual_integral_threshold": 20,
            "step1_residual_integral_change_start_point": False,
            "step1_residual_integral_change_start_point_n_sigma": 3,
            "step2_clustering_method_name": "dbscan",
            "step2_dbscan_min_pts": 1,
            "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
            "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        }, 
        {
            "enable_unireducer": False,
            "enable_multireducer": True,
            "step1_method_name": "residual_integral",
            "step2_clustering_method_name": "dbscan",
            "step2_dbscan_min_pts": 1,
            "step2_dbscan_dist_type": "sbd",  # 'pearsonr' or 'sbd'
            "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        }, {
            "enable_unireducer": True,
            "enable_multireducer": True,
            "step1_method_name": "residual_integral",
            "step1_residual_integral_threshold": 20,
            "step1_residual_integral_change_start_point": False,
            "step1_residual_integral_change_start_point_n_sigma": 3,
            "step2_clustering_method_name": "dbscan",
            "step2_dbscan_min_pts": 1,
            "step2_dbscan_dist_type": "sbd",  # 'pearsonr' or 'sbd'
            "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        }
    ],
)

/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/notebooks/dataset/../../eval/tsdr.py:109: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(project=os.environ["TSDR_NEPTUNE_PROJECT"])


https://app.neptune.ai/yuuk1/tsdr/e/TSDR-115
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 17 operations to synchronize with Neptune. Do not kill this process.
All 17 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-115/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-116
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 17 operations to synchronize with Neptune. Do not kill this process.
All 17 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-116/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-117
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 18 operations to synchronize with Neptune. Do not kill this process.
All 18 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1

/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by dat

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 17 operations to synchronize with Neptune. Do not kill this process.
All 17 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-135/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-136


/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by dat

...

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 17 operations to synchronize with Neptune. Do not kill this process.
All 17 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-144/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-145


/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by dat

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 18 operations to synchronize with Neptune. Do not kill this process.
All 18 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-145/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-146


/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(
/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:206: EfficiencyWarning: Precomputed sparse input was not sorted by dat

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 18 operations to synchronize with Neptune. Do not kill this process.
All 18 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-146/metadata
